In [10]:
"""
Flight Route Weather Prediction System
======================================
MAIN TASK: Barycentric Interpolation
Uses Inverse Distance Weighting (IDW), a form of generalized barycentric
interpolation, to predict weather along complex flight paths.

Input: Source station ID + Destination station ID (used to define the route)
Output: Interpolated weather predictions at waypoints along the route
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import math # Import math for floor operation
warnings.filterwarnings('ignore')

class FlightWeatherPredictor:
    """
    Primary Class: Predicts weather conditions along flight routes using barycentric interpolation.
    """

    # --- UPDATED DEFAULT FILE NAME FOR CONSISTENCY ---
    def __init__(self, data_path='merged_file_cleaned.csv'):
        """
        Initialize predictor with training data

        Args:
            data_path: Path to your CSV file with weather data
        """
        print("Loading training data...")

        # --- FIX 1: Robust Data Loading (Checking Comma then Semicolon) ---
        try:
            # Try comma delimiter (common for uploaded CSVs)
            self.data = pd.read_csv(data_path, sep=',')
        except pd.errors.ParserError:
            try:
                # Fallback to semicolon delimiter
                self.data = pd.read_csv(data_path, sep=';')
            except Exception as e:
                print(f"FATAL ERROR: Failed to load data. Check delimiter and file format. Error: {e}")
                raise

        # --- FIX 2: Data Cleaning for Coordinates and Station ID ---
        print("Cleaning and standardizing data columns...")

        # Standardize station ID to uppercase string (important for lookups)
        if 'station' in self.data.columns:
            self.data['station'] = self.data['station'].astype(str).str.upper()
        if 'Station' in self.data.columns:
             self.data.rename(columns={'Station': 'station'}, inplace=True)


        # Convert latitude/longitude to float and de-scale (assuming they were multiplied by 10^6)
        if 'latitude' in self.data.columns and 'longitude' in self.data.columns:
            # Convert to numeric, coercing errors to NaN
            self.data['latitude'] = pd.to_numeric(self.data['latitude'], errors='coerce')
            self.data['longitude'] = pd.to_numeric(self.data['longitude'], errors='coerce')

            # De-scale the coordinates
            if (self.data['latitude'].abs() > 1000).any() and (self.data['latitude'].abs() < 90000000).any():
                print("Detected scaled coordinates (e.g., 491325). De-scaling by 1,000,000...")
                self.data['latitude'] /= 1000000.0
                self.data['longitude'] /= 1000000.0

        # --- FIX 3: Altitude Handling (NO CONVERSION APPLIED) ---
        # The raw values from 'altitude_surface_ft' are used directly.
        ALT_COL_RAW = 'altitude_surface_ft'
        if ALT_COL_RAW in self.data.columns:
            print(f"Using raw values from '{ALT_COL_RAW}' (assumed to be in feet, NO conversion applied)...")
            self.data[ALT_COL_RAW] = pd.to_numeric(self.data[ALT_COL_RAW], errors='coerce')
        # The conversion line (self.data[ALT_COL_RAW] = self.data[ALT_COL_RAW] * 3.28084) is REMOVED.

        # ------------------------------------------------------------------
        # --- DATA IMPUTATION: Handle missing cloud ceiling data ---
        # ------------------------------------------------------------------
        # The correct column name used in the CSV is 'ceiling_feet' not 'cloudcelings'
        CLOUD_CEILING_COL = 'ceiling_feet'
        self.NO_CEILING_VALUE = 50000.0 # Store this for use in get_station_weather

        if CLOUD_CEILING_COL in self.data.columns:
            print(f"Applying imputation to '{CLOUD_CEILING_COL}' to handle missing data...")

            # 1. Convert column to numeric, forcing errors (like 'CLR', 'N/A') to NaN
            self.data[CLOUD_CEILING_COL] = pd.to_numeric(
                self.data[CLOUD_CEILING_COL], errors='coerce'
            )

            # 2. Impute NaN/missing values with the high number (ensures 3 valid inputs)
            self.data[CLOUD_CEILING_COL].fillna(self.NO_CEILING_VALUE, inplace=True)

        # Drop any rows where coordinates are still missing or invalid after cleaning
        self.data.dropna(subset=['latitude', 'longitude'], inplace=True)

        # Display dataset structure
        print(f"\nDataset loaded: {len(self.data)} records")

        # Extract unique stations
        self.stations = self._extract_station_info()
        print(f"\n✓ Found {len(self.stations)} unique weather stations")

    def _extract_station_info(self):
        """
        Extract station information from the dataset
        """
        # Try common column name variations
        possible_id_cols = ['station_id', 'ICAO', 'icao', 'station', 'aerodrome']
        possible_lat_cols = ['latitude', 'lat', 'Latitude']
        possible_lon_cols = ['longitude', 'lon', 'long', 'Longitude']

        # --- FIX 4: Add actual altitude column name for better detection ---
        possible_alt_cols = ['altitude', 'elevation', 'alt', 'Altitude', 'altitude_surface_ft']

        # Find actual column names
        id_col = next((col for col in possible_id_cols if col in self.data.columns), None)
        lat_col = next((col for col in possible_lat_cols if col in self.data.columns), None)
        lon_col = next((col for col in possible_lon_cols if col in self.data.columns), None)
        alt_col = next((col for col in possible_alt_cols if col in self.data.columns), None)

        if not all([id_col, lat_col, lon_col]):
            print("\n Warning: Could not auto-detect primary column names.")
            print("Available columns:", list(self.data.columns))
            raise ValueError("Required columns (station ID, lat, lon) not found or specified.")

        # Extract unique stations with their coordinates
        if alt_col and alt_col in self.data.columns:
            stations = self.data[[id_col, lat_col, lon_col, alt_col]].drop_duplicates(subset=[id_col])
            stations.columns = ['station_id', 'latitude', 'longitude', 'altitude']
        else:
            stations = self.data[[id_col, lat_col, lon_col]].drop_duplicates(subset=[id_col])
            stations.columns = ['station_id', 'latitude', 'longitude']
            # Removed invalid non-printable character U+00A0
            stations['altitude'] = 0 # Default altitude if not available

        # Ensure coordinates are numeric before setting index
        stations['latitude'] = pd.to_numeric(stations['latitude'], errors='coerce')
        stations['longitude'] = pd.to_numeric(stations['longitude'], errors='coerce')

        return stations.set_index('station_id')

    # --- FIX 5: Use robust formatting method (to handle NaN) ---
    def list_available_stations(self):
        """
        Display all available stations in the requested format, ensuring altitude
        is the raw value from the dataset.
        """
        print("\n" + "="*70)
        print(f"AVAILABLE WEATHER STATIONS (First 10 of {len(self.stations)} Found)")
        print("="*70)

        stations_to_show = self.stations.copy()

        # Create columns matching the user's requested output format
        stations_to_show['Latitude'] = stations_to_show['latitude'].apply(lambda x: f"{x:.4f}")
        stations_to_show['Longitude'] = stations_to_show['longitude'].apply(lambda x: f"{x:.4f}")

        # Altitude uses the original value, flooring it for display if not NaN
        stations_to_show['Altitude (ft)'] = stations_to_show['altitude'].apply(
            lambda x: f"{math.floor(x)}" if pd.notna(x) else "N/A"
        )

        # Final display table (Showing the top 10 as a sample)
        df_display = stations_to_show[['Latitude', 'Longitude', 'Altitude (ft)']].head(10)
        print(df_display.to_string(header=True, index=True, index_names=['Station ID']))

        if len(self.stations) > 10:
            print(f"\n... and {len(self.stations) - 10} more stations. (Total: {len(self.stations)})")

        print("="*70)

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """Calculate great circle distance in kilometers"""
        # FIX: Removed invalid non-printable character U+00A0
        R = 6371.0 # Earth radius in km
        lat1_rad, lat2_rad = np.radians(lat1), np.radians(lat2)
        delta_lat = np.radians(lat2 - lat1)
        delta_lon = np.radians(lon2 - lon1)

        a = (np.sin(delta_lat / 2) ** 2 +
             np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon / 2) ** 2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        return R * c

    def find_nearest_stations(self, lat, lon, n=3):
        """
        Find n nearest weather stations to given coordinates (input for barycentric calculation)

        Returns:
            DataFrame with nearest stations and their distances
        """
        distances = []
        for station_id, row in self.stations.iterrows():
            dist = self.haversine_distance(lat, lon, row['latitude'], row['longitude'])
            distances.append({
                'station_id': station_id,
                'distance_km': dist,
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'altitude': row['altitude']
            })

        df = pd.DataFrame(distances).sort_values('distance_km')
        return df.head(n)

    def get_station_weather(self, station_id, timestamp=None):
        """
        Get most recent weather observation for a station (data source for interpolation)
        """
        # Find column name for station ID
        id_col = [col for col in self.data.columns if 'station' in col.lower() or 'icao' in col.lower()][0]

        # Filter data for this station
        station_data = self.data[self.data[id_col] == station_id]

        if len(station_data) == 0:
            return None

        # Get most recent observation (or use timestamp if provided)
        if timestamp is None:
            latest = station_data.iloc[-1]
        else:
            # Simplified: just use most recent for now
            latest = station_data.iloc[-1]

        # Extract weather parameters (adjust column names based on your data)
        weather = {}

        # Common weather parameter column names (updated to match CSV data)
        param_mappings = {
            'temperature': ['temperature_c', 'temperature', 'temp', 'Temperature', 'temp_c'],
            'pressure': ['pressure_hpa', 'pressure', 'press', 'Pressure'],
            'wind_speed': ['wind_speed_kt', 'wind_speed', 'windspeed', 'Wind_Speed', 'wspd'],
            'wind_direction': ['wind_direction_deg', 'wind_direction', 'wind_dir', 'Wind_Direction', 'wdir'],
            'visibility': ['visibility_sm', 'visibility', 'vis', 'Visibility'],
            # The corrected column name is used here:
            'cloud_ceiling': ['ceiling_feet', 'cloud_ceiling', 'cloudcelings', 'Cloud_Ceiling', 'clouds']
        }

        for param, possible_cols in param_mappings.items():
            col = next((c for c in possible_cols if c in self.data.columns), None)
            if col:
                # Use .get() for safety and retrieve the cleaned numeric value
                value = latest.get(col)

                # Check for NaT/None, but rely mostly on the imputation done in __init__
                if pd.isna(value) and param == 'cloud_ceiling':
                    # Fallback to the high value if missing, ensuring interpolation works
                    weather[param] = self.NO_CEILING_VALUE
                else:
                    weather[param] = value
            else:
                weather[param] = None

        return weather

    def interpolate_circular(self, angles, weights):
        """
        Critical step for barycentric interpolation of circular data (wind direction)
        Handles the 360/0 degree crossover.
        """
        angles_rad = np.radians(angles)
        x = np.sum(weights * np.cos(angles_rad))
        y = np.sum(weights * np.sin(angles_rad))
        result = np.degrees(np.arctan2(y, x))
        return result % 360

    # --- MODIFICATION: Added n_neighbors parameter (Re-introduced) ---
    def interpolate_weather(self, lat, lon, altitude=None, n_neighbors=3):
        """
        The **Core Barycentric Interpolation** method (Inverse Distance Weighting).
        Now uses 'n_neighbors' stations for greater flexibility.
        """
        # Find n_neighbors nearest stations
        nearest = self.find_nearest_stations(lat, lon, n=n_neighbors)

        # Calculate inverse distance weights (barycentric step)
        # FIX: Removed invalid non-printable character U+00A0
        epsilon = 0.001 # Avoid division by zero
        inv_distances = 1.0 / (nearest['distance_km'].values + epsilon) ** 2
        weights = inv_distances / np.sum(inv_distances) # These are the generalized barycentric weights

        # Get weather data from each station
        weather_data = []
        for station_id in nearest['station_id']:
            weather = self.get_station_weather(station_id)
            if weather:
                weather_data.append(weather)

        # Check if enough stations were found (now checks against n_neighbors)
        if len(weather_data) < n_neighbors:
            # Return None to signal failure, handled gracefully by main()
            return None

        # Interpolate each parameter
        result = {
            'latitude': lat,
            'longitude': lon,
            'altitude': altitude if altitude else 0
        }

        # Linear parameters
        for param in ['temperature', 'pressure', 'wind_speed', 'visibility', 'cloud_ceiling']:
            # The check now ensures we have n_neighbors valid values
            values = [w[param] for w in weather_data if w[param] is not None and pd.notna(w[param])]
            if len(values) == n_neighbors:
                # Core IDW calculation
                result[param] = np.sum(weights * np.array(values))
            else:
                result[param] = None


        # Circular parameter (wind direction)
        wind_dirs = [w['wind_direction'] for w in weather_data if w['wind_direction'] is not None and pd.notna(w['wind_direction'])]
        if len(wind_dirs) == n_neighbors:
            result['wind_direction'] = self.interpolate_circular(np.array(wind_dirs), weights)
        else:
            result['wind_direction'] = None

        # Altitude correction (if target altitude is different)
        if altitude and altitude > 0:
            avg_station_alt = nearest['altitude'].mean()
            altitude_diff_ft = altitude - avg_station_alt

            # Temperature lapse rate: -1.98°C per 1000 ft
            if result['temperature'] is not None:
                result['temperature'] += -1.98 * (altitude_diff_ft / 1000.0)

            # Pressure: approximately -1 hPa per 30 ft near sea level (simplistic)
            if result['pressure'] is not None:
                result['pressure'] += -1.0 * (altitude_diff_ft / 30.0)

        # Add metadata
        result['nearest_stations'] = nearest['station_id'].tolist()
        result['station_distances'] = nearest['distance_km'].tolist()
        result['interpolation_weights'] = weights.tolist()

        return result

    def generate_route_waypoints(self, source_id, dest_id, num_waypoints=10):
        """
        Generate waypoints along route from source to destination (Feeds the interpolator)
        """
        if source_id not in self.stations.index:
            raise ValueError(f"Source station '{source_id}' not found")
        if dest_id not in self.stations.index:
            raise ValueError(f"Destination station '{dest_id}' not found")

        source = self.stations.loc[source_id]
        dest = self.stations.loc[dest_id]

        # Calculate route distance
        distance_km = self.haversine_distance(
            source['latitude'], source['longitude'],
            dest['latitude'], dest['longitude']
        )

        # Generate waypoints (linear interpolation for simplicity)
        waypoints = []
        for i in range(num_waypoints + 2): # Include source and destination
            fraction = i / (num_waypoints + 1)

            lat = source['latitude'] + fraction * (dest['latitude'] - source['latitude'])
            lon = source['longitude'] + fraction * (dest['longitude'] - source['longitude'])

            # Simulate cruise altitude profile (climb, cruise, descend)
            CRUISE_ALT = 25000 # Example cruise altitude in ft
            CLIMB_FRAC = 0.2
            DESCENT_FRAC = 0.8

            if fraction < CLIMB_FRAC: # Climb phase
                altitude = source['altitude'] + (fraction / CLIMB_FRAC) * (CRUISE_ALT - source['altitude'])
            elif fraction < DESCENT_FRAC: # Cruise phase
                altitude = CRUISE_ALT
            else: # Descent phase
                altitude = CRUISE_ALT - ((fraction - DESCENT_FRAC) / (1 - DESCENT_FRAC)) * (CRUISE_ALT - dest['altitude'])

            waypoints.append({
                'waypoint_number': i + 1,
                'latitude': lat,
                'longitude': lon,
                'altitude': altitude,
                'distance_from_source': distance_km * fraction
            })

        return waypoints, distance_km

    # --- MODIFICATION: Added n_stations parameter (Re-introduced) ---
    def predict_route_weather(self, source_id, dest_id, num_waypoints=10, n_stations=3):
        """
        Predict weather along entire flight route by iteratively calling the
        barycentric interpolator for each waypoint.
        """
        print(f"\n{'='*80}")
        print(f"FLIGHT ROUTE WEATHER PREDICTION")
        print(f"{'='*80}")
        print(f"Route: {source_id} → {dest_id}")

        # Generate waypoints
        waypoints, total_distance = self.generate_route_waypoints(source_id, dest_id, num_waypoints)
        print(f"Total Distance: {total_distance:.1f} km")
        print(f"Number of Waypoints: {len(waypoints)}")
        print(f"Using N={n_stations} nearest stations for interpolation.") # Added print
        print(f"\n{'='*80}")

        # Predict weather at each waypoint
        predictions = []
        for wp in waypoints:
            # --- CORE TASK EXECUTION ---
            weather = self.interpolate_weather(
                wp['latitude'],
                wp['longitude'],
                wp['altitude'],
                n_neighbors=n_stations # Passed the new parameter
            )

            if weather:
                predictions.append({
                    **wp,
                    **{k: v for k, v in weather.items()
                        if k not in ['latitude', 'longitude', 'altitude']}
                })

        return pd.DataFrame(predictions)


def format_weather_display(predictions_df):
    """
    Format predictions for nice display
    """
    if predictions_df.empty:
        print("\nNo weather predictions available.")
        return

    print("\n" + "="*150)
    print("WEATHER FORECAST ALONG ROUTE")
    print("="*150)

    # Display summary columns
    display_cols = ['waypoint_number', 'distance_from_source', 'altitude',
                    'temperature', 'pressure', 'wind_direction', 'wind_speed',
                    'visibility', 'cloud_ceiling']

    # Filter to only columns that exist
    display_cols = [col for col in display_cols if col in predictions_df.columns]

    df_display = predictions_df[display_cols].copy()

    # Format columns
    if 'distance_from_source' in df_display.columns:
        df_display['distance_km'] = df_display['distance_from_source'].apply(lambda x: f"{x:.0f}km" if pd.notna(x) else "N/A")
    if 'altitude' in df_display.columns:
        df_display['altitude_ft'] = df_display['altitude'].apply(lambda x: f"{x:.0f}ft" if pd.notna(x) else "N/A")
    if 'temperature' in df_display.columns:
        df_display['temp_c'] = df_display['temperature'].apply(lambda x: f"{x:.1f}°C" if pd.notna(x) else "N/A")
    if 'pressure' in df_display.columns:
        df_display['pressure_hpa'] = df_display['pressure'].apply(lambda x: f"{x:.1f} hPa" if pd.notna(x) else "N/A")
    if 'wind_direction' in df_display.columns and 'wind_speed' in df_display.columns:
        df_display['wind'] = df_display.apply(
            lambda row: f"{row['wind_direction']:.0f}°@{row['wind_speed']:.0f}kt"
            if pd.notna(row['wind_direction']) and pd.notna(row['wind_speed']) else "N/A",
            axis=1
        )
    if 'visibility' in df_display.columns:
        df_display['vis_sm'] = df_display['visibility'].apply(lambda x: f"{x:.1f} SM" if pd.notna(x) else "N/A")

    if 'cloud_ceiling' in df_display.columns:
        # Display the numerical value of the ceiling in feet, including the imputed high value.
        df_display['ceiling'] = df_display['cloud_ceiling'].apply(lambda x: f"{x:.0f} ft" if pd.notna(x) else "N/A")

    # Select final display columns
    final_cols = ['waypoint_number', 'distance_km', 'altitude_ft', 'temp_c', 'pressure_hpa', 'wind', 'vis_sm', 'ceiling']
    final_cols = [col for col in final_cols if col in df_display.columns]

    print(df_display[final_cols].to_string(index=False))
    print("="*150)

    # Show interpolation details for a few waypoints
    print("\nINTERPOLATION DETAILS (Sample Waypoints):")
    print("-"*150)
    # Use max(1, len(predictions_df)//2) to avoid IndexError on small DFs
    sample_indices = [0, max(1, len(predictions_df)//2), len(predictions_df)-1]

    for idx in sample_indices:
        if idx < len(predictions_df):
            row = predictions_df.iloc[idx]
            print(f"\nWaypoint {row['waypoint_number']}: ({row['latitude']:.3f}, {row['longitude']:.3f}) @ {row['altitude']:.0f}ft")
            if 'nearest_stations' in row:
                print(f"  Nearest stations: {row['nearest_stations']}")
                if 'station_distances' in row:
                    print(f"  Distances: {[f'{d:.1f}km' for d in row['station_distances']]}")
                if 'interpolation_weights' in row:
                    print(f"  Weights: {[f'{w:.3f}' for w in row['interpolation_weights']]}")
    print("="*150)


def main():
    """
    Main function - Interactive weather prediction system
    """
    # Removed invalid non-printable characters U+00A0
    print("     FLIGHT ROUTE WEATHER PREDICTION SYSTEM")
    print("     Using Barycentric Interpolation")

    # Initialize predictor
    try:
        # --- FIX 6: Use the correct file name for the provided data ---
        predictor = FlightWeatherPredictor('merged_file_cleaned.csv')
    except Exception as e:
        print(f"\n FAILED TO INITIALIZE PREDICTOR: {e}")
        return

    while True:
        print("\n" + "="*80)
        print("MAIN MENU")
        print("="*80)
        print("1. Predict weather along flight route")
        print("2. Predict weather at custom location")
        print("3. Exit")
        print("="*80)

        choice = input("\nEnter your choice (1-3): ").strip()

        if choice == '1':
            print("\n--- ROUTE WEATHER PREDICTION ---")
            # Added listing stations here
            predictor.list_available_stations()

            source_id = input("\nEnter source station ID (e.g., CYVZ): ").strip().upper()
            dest_id = input("Enter destination station ID (e.g., CYBB): ").strip().upper()

            try:
                # Robust Station ID Check
                if source_id not in predictor.stations.index:
                    raise ValueError(f"Source station '{source_id}' not found in the dataset.")
                if dest_id not in predictor.stations.index:
                    raise ValueError(f"Destination station '{dest_id}' not found in the dataset.")

                num_waypoints = int(input("Number of intermediate waypoints you want to create (default 10): ") or "10")
            except ValueError as ve:
                print(f"\n Error in input: {ve}")
                continue
            except:
                num_waypoints = 10

            # --- MODIFICATION: New user input for N (Re-introduced) ---
            try:
                n_stations = int(input("Number of nearest stations (N) for interpolation (default 3, try 5): ") or "3")
                if n_stations < 1: n_stations = 3
            except:
                n_stations = 3
            # ----------------------------------------

            try:
                # Pass n_stations to the prediction function
                predictions = predictor.predict_route_weather(source_id, dest_id, num_waypoints, n_stations)
                format_weather_display(predictions)

            except Exception as e:
                print(f"\n Error during prediction: {e}")

        elif choice == '2':
            print("\n--- CUSTOM LOCATION PREDICTION ---")
            try:
                lat = float(input("Enter latitude: "))
                lon = float(input("Enter longitude: "))
                alt = float(input("Enter altitude (ft, press Enter for surface): ") or "0")

                # --- MODIFICATION: New user input for N (Re-introduced) ---
                try:
                    n_stations = int(input("Number of nearest stations (N) for interpolation (default 3, try 5): ") or "3")
                    if n_stations < 1: n_stations = 3
                except:
                    n_stations = 3
                # ----------------------------------------

                # Pass n_stations to the interpolation function
                weather = predictor.interpolate_weather(lat, lon, alt, n_neighbors=n_stations)

                if weather:
                    print("\n" + "="*80)
                    print("PREDICTED WEATHER")
                    print("="*80)
                    print(f"Location: ({lat:.4f}, {lon:.4f}) @ {alt:.0f} ft")
                    print(f"Interpolation used N={n_stations} nearest stations.")

                    # --- REVISED FIX: Robust Formatting for None/NaN values (Applied the fix) ---
                    temp = weather.get('temperature')
                    press = weather.get('pressure')
                    wdir = weather.get('wind_direction')
                    wspd = weather.get('wind_speed')
                    vis = weather.get('visibility')

                    # Explicitly create display string only if not NaN
                    temp_display = 'N/A'
                    if pd.notna(temp):
                        temp_display = f"{temp:.1f}°C"

                    press_display = 'N/A'
                    if pd.notna(press):
                        press_display = f"{press:.1f} hPa"

                    print(f"\nTemperature: {temp_display}")
                    print(f"Pressure: {press_display}")

                    # Combine wind direction and speed
                    wind_str = 'N/A'
                    if pd.notna(wdir) and pd.notna(wspd):
                        wind_str = f"{wdir:.0f}° @ {wspd:.0f} kt"
                    print(f"Wind: {wind_str}")

                    vis_display = 'N/A'
                    if pd.notna(vis):
                        vis_display = f"{vis:.1f} SM"
                    print(f"Visibility: {vis_display}")
                    # ----------------------------------------------------

                    ceiling_val = weather.get('cloud_ceiling', None)
                    if pd.notna(ceiling_val):
                        # Display the numerical value of the ceiling in feet.
                        display_text = f"{ceiling_val:.0f} ft AGL"
                        print(f"Cloud Ceiling: {display_text}")
                    else:
                        print("Cloud Ceiling: N/A")

                    print(f"\nNearest Stations: {weather.get('nearest_stations', [])}")
                    print(f"Distances: {[f'{d:.1f}km' for d in weather.get('station_distances', [])]}")
                    print(f"Weights: {[f'{w:.3f}' for w in weather.get('interpolation_weights', [])]}")
                    print("="*80)
                else:
                    # Added N to the failure message for clarity
                    print(f" Could not predict weather at this location (Need {n_stations} valid stations or a nearby station in the dataset)")
                    # Added a note for extreme/impossible coordinates
                    if not (-90 <= lat <= 90 and -180 <= lon <= 180):
                        print("\n Note: Check your coordinates. Latitude must be between -90 and +90.")


            except Exception as e:
                print(f"\n Error: {e}")

        elif choice == '3':
            print("\n Thank you for using the Flight Weather Prediction System!")
            # Removed invalid non-printable character U+00A0
            print(" Safe flights! \n")
            break

        else:
            print(" Invalid choice. Please enter 1-3.")


if __name__ == "__main__":
    main()

     FLIGHT ROUTE WEATHER PREDICTION SYSTEM
     Using Barycentric Interpolation
Loading training data...
Cleaning and standardizing data columns...
Using raw values from 'altitude_surface_ft' (assumed to be in feet, NO conversion applied)...
Applying imputation to 'ceiling_feet' to handle missing data...

Dataset loaded: 82487 records

✓ Found 231 unique weather stations

MAIN MENU
1. Predict weather along flight route
2. Predict weather at custom location
3. Exit

Enter your choice (1-3): 1

--- ROUTE WEATHER PREDICTION ---

AVAILABLE WEATHER STATIONS (First 10 of 231 Found)
           Latitude  Longitude Altitude (ft)
station_id                                  
CYBB        68.5357   -89.8055            56
CYBC        49.1325   -68.2044            71
CYBD        52.3875  -126.5960           117
CYBG        48.3319   -70.9929           522
CYBK        64.2989   -96.0778            59
CYBL        49.9508  -125.2710           346
CYBQ        58.7063   -98.5111           923
CYBR       